### Adjust Word Group In Youtube Sentence Word Usage Analysis

In [1]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [3]:
# language pair
lang_folder = "Dutch"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# native word select for Part 2 
word_start = 0  # 0  # native word start index
word_end = 200 # 200 # native word end index

# word all usage in sent
word_use_num_max = 7  # 5
word_use_num_min = 1  # 1

# youtube sentence usage for remain word 
youtube_sent_usage = "selected"  # "all","selected"

In [4]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Talk Time/Result/2-Adjust Word Group In Youtube Sentence Word Usage Analysis"

Path(path).mkdir(parents=True, exist_ok=True)

In [5]:
def word_in_wordgroup_simple(source_word_list, df_target, target_column, word_sample_num):

    '''word_in_wordgroup_simple(not_in_sent_word_list, df_youtube_sent_select, "search_string", 5)\n
       source_word_list is searching word list\n
       df_target is dataframe, target_column are dataframe column string name\n
       word_sample_num is searching sample number. 
    '''
    
    df_select = df_target[[f"{target_column}"]].dropna()
    df_result = pd.DataFrame()
    for i in source_word_list:
        try:
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].head(word_sample_num)    
        except:
            pass        
        word_in_word_cluster.insert(0,"word",i)
        df_result = pd.concat([df_result,word_in_word_cluster], axis=0)
    df_result.reset_index(drop=True, inplace=True)

    return df_result

#### Word Usage

In [6]:
def word_index(df_word, word_column, sentence):
    word_index_list = []
    word_index_dict = {var2:var1 for (var1, var2) in enumerate(df_word[f"{word_column}"])}
    words = word_tokenize(sentence)
    for word in words:
        value = word_index_dict[f"{word}"]
        word_index_list.append(value)
    return word_index_list

In [7]:
disable_video_id_list = ["H6E6N70jYqI","BoaYsdPtJYA","HUZINsU40Fk","Et3diPcEmfY"]

In [8]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,ik,22539531
1,je,20769946
2,het,15696161
3,de,15258816
4,dat,13387137
...,...,...
331769,oorlogsgezichten,5
331770,opdrachtenlijst,5
331771,verlsaafde,5
331772,oxidatieproces,5


In [9]:
df_word_select = df_word_all.iloc[word_start:word_end,]
df_word_select.reset_index(inplace=True)
df_word_select

,index,word,frequency
0,0,ik,22539531
1,1,je,20769946
2,2,het,15696161
3,3,de,15258816
4,4,dat,13387137
...,...,...,...
195,195,genoeg,353128
196,196,wist,352948
197,197,oh,351988
198,198,klaar,350506


In [10]:
df_youtube_sent_result = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Talk Time/Data/\
1-Word Group In Youtube Sentence/{lang_folder.lower().capitalize()}_Word_Group_Max_In_Youtube_Sentence_{word_end}_Word.csv")
df_youtube_sent_result

,search_string,start_time,end_time,sentence,video_id,video_url
0,en n z,446,448,en daarmee speeldn we e oorlogje o e zeeslage ...,P8gWZjlPaNo,https://www.youtube.com/watch?v=P8gWZjlPaNo&t=...
1,ga ik u,40,41,en zoals gezegd in de inleiding ga ik u vandaa...,Gokx4ZpGRC0,https://www.youtube.com/watch?v=Gokx4ZpGRC0&t=40s
2,en de n,6257,6259,uit vloeit en de n,R4Q1cNbNKU0,https://www.youtube.com/watch?v=R4Q1cNbNKU0&t=...
3,de m te,253,255,vorig jaar deed de m te vormen op de groepsfot...,HNqNQNMXio4,https://www.youtube.com/watch?v=HNqNQNMXio4&t=...
4,u op de,107,109,vervolgens drukt u op de neertoets en de optoe...,DEHw1VDSKVc,https://www.youtube.com/watch?v=DEHw1VDSKVc&t=...
...,...,...,...,...,...,...
173324,dat er voor ja moet er geen gaan we nou ja maa...,694,700,jarenlang dat er voor ja moet er geen gaan we ...,9rsOGla4V6s,https://www.youtube.com/watch?v=9rsOGla4V6s&t=...
173325,en de huis gaan en ik hou gewoon heel erg veel...,207,213,leger en de huis gaan en ik hou gewoon heel er...,V1RX2oPECHY,https://www.youtube.com/watch?v=V1RX2oPECHY&t=...
173326,als ik het niet ben ik zeggen jaar dan heb jij...,2017,2024,lieg als ik het niet ben ik zeggen jaar dan he...,iLJg6UP0y_8,https://www.youtube.com/watch?v=iLJg6UP0y_8&t=...
173327,niet zeggen denk en ik het goed gedaan maar di...,742,746,niet zeggen denk en ik het goed gedaan maar di...,Pg4VBpojtGc,https://www.youtube.com/watch?v=Pg4VBpojtGc&t=...


In [11]:
df_youtube_sent_result["word_index_list"] = df_youtube_sent_result["search_string"].apply(lambda sent: word_index(df_word_select, "word", sent))
df_youtube_sent_result

,search_string,start_time,end_time,sentence,video_id,video_url,word_index_list
0,en n z,446,448,en daarmee speeldn we e oorlogje o e zeeslage ...,P8gWZjlPaNo,https://www.youtube.com/watch?v=P8gWZjlPaNo&t=...,"[8, 26, 139]"
1,ga ik u,40,41,en zoals gezegd in de inleiding ga ik u vandaa...,Gokx4ZpGRC0,https://www.youtube.com/watch?v=Gokx4ZpGRC0&t=40s,"[60, 0, 33]"
2,en de n,6257,6259,uit vloeit en de n,R4Q1cNbNKU0,https://www.youtube.com/watch?v=R4Q1cNbNKU0&t=...,"[8, 3, 26]"
3,de m te,253,255,vorig jaar deed de m te vormen op de groepsfot...,HNqNQNMXio4,https://www.youtube.com/watch?v=HNqNQNMXio4&t=...,"[3, 71, 15]"
4,u op de,107,109,vervolgens drukt u op de neertoets en de optoe...,DEHw1VDSKVc,https://www.youtube.com/watch?v=DEHw1VDSKVc&t=...,"[33, 17, 3]"
...,...,...,...,...,...,...,...
173324,dat er voor ja moet er geen gaan we nou ja maa...,694,700,jarenlang dat er voor ja moet er geen gaan we ...,9rsOGla4V6s,https://www.youtube.com/watch?v=9rsOGla4V6s&t=...,"[4, 19, 21, 48, 44, 19, 41, 63, 11, 106, 48, 1..."
173325,en de huis gaan en ik hou gewoon heel erg veel...,207,213,leger en de huis gaan en ik hou gewoon heel er...,V1RX2oPECHY,https://www.youtube.com/watch?v=V1RX2oPECHY&t=...,"[8, 3, 153, 63, 8, 0, 128, 121, 141, 174, 113,..."
173326,als ik het niet ben ik zeggen jaar dan heb jij...,2017,2024,lieg als ik het niet ben ik zeggen jaar dan he...,iLJg6UP0y_8,https://www.youtube.com/watch?v=iLJg6UP0y_8&t=...,"[25, 0, 2, 7, 28, 0, 126, 150, 35, 24, 38, 2, ..."
173327,niet zeggen denk en ik het goed gedaan maar di...,742,746,niet zeggen denk en ik het goed gedaan maar di...,Pg4VBpojtGc,https://www.youtube.com/watch?v=Pg4VBpojtGc&t=...,"[7, 126, 90, 8, 0, 2, 50, 142, 18, 30, 39, 0, ..."


In [12]:
df_youtube_sent_result = df_youtube_sent_result[~df_youtube_sent_result["video_id"].isin(disable_video_id_list)]
df_youtube_sent_result

,search_string,start_time,end_time,sentence,video_id,video_url,word_index_list
0,en n z,446,448,en daarmee speeldn we e oorlogje o e zeeslage ...,P8gWZjlPaNo,https://www.youtube.com/watch?v=P8gWZjlPaNo&t=...,"[8, 26, 139]"
1,ga ik u,40,41,en zoals gezegd in de inleiding ga ik u vandaa...,Gokx4ZpGRC0,https://www.youtube.com/watch?v=Gokx4ZpGRC0&t=40s,"[60, 0, 33]"
2,en de n,6257,6259,uit vloeit en de n,R4Q1cNbNKU0,https://www.youtube.com/watch?v=R4Q1cNbNKU0&t=...,"[8, 3, 26]"
3,de m te,253,255,vorig jaar deed de m te vormen op de groepsfot...,HNqNQNMXio4,https://www.youtube.com/watch?v=HNqNQNMXio4&t=...,"[3, 71, 15]"
4,u op de,107,109,vervolgens drukt u op de neertoets en de optoe...,DEHw1VDSKVc,https://www.youtube.com/watch?v=DEHw1VDSKVc&t=...,"[33, 17, 3]"
...,...,...,...,...,...,...,...
173324,dat er voor ja moet er geen gaan we nou ja maa...,694,700,jarenlang dat er voor ja moet er geen gaan we ...,9rsOGla4V6s,https://www.youtube.com/watch?v=9rsOGla4V6s&t=...,"[4, 19, 21, 48, 44, 19, 41, 63, 11, 106, 48, 1..."
173325,en de huis gaan en ik hou gewoon heel erg veel...,207,213,leger en de huis gaan en ik hou gewoon heel er...,V1RX2oPECHY,https://www.youtube.com/watch?v=V1RX2oPECHY&t=...,"[8, 3, 153, 63, 8, 0, 128, 121, 141, 174, 113,..."
173326,als ik het niet ben ik zeggen jaar dan heb jij...,2017,2024,lieg als ik het niet ben ik zeggen jaar dan he...,iLJg6UP0y_8,https://www.youtube.com/watch?v=iLJg6UP0y_8&t=...,"[25, 0, 2, 7, 28, 0, 126, 150, 35, 24, 38, 2, ..."
173327,niet zeggen denk en ik het goed gedaan maar di...,742,746,niet zeggen denk en ik het goed gedaan maar di...,Pg4VBpojtGc,https://www.youtube.com/watch?v=Pg4VBpojtGc&t=...,"[7, 126, 90, 8, 0, 2, 50, 142, 18, 30, 39, 0, ..."


In [13]:
#df_youtube_sent_select.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Group_In_Youtube_Sentence_Word_Index_List.xlsx", index=False)

In [14]:
df_youtube_sent_select = df_youtube_sent_result[df_youtube_sent_result["start_time"] <= 700]
df_youtube_sent_select.reset_index(drop=True, inplace=True)
df_youtube_sent_select

,search_string,start_time,end_time,sentence,video_id,video_url,word_index_list
0,en n z,446,448,en daarmee speeldn we e oorlogje o e zeeslage ...,P8gWZjlPaNo,https://www.youtube.com/watch?v=P8gWZjlPaNo&t=...,"[8, 26, 139]"
1,ga ik u,40,41,en zoals gezegd in de inleiding ga ik u vandaa...,Gokx4ZpGRC0,https://www.youtube.com/watch?v=Gokx4ZpGRC0&t=40s,"[60, 0, 33]"
2,de m te,253,255,vorig jaar deed de m te vormen op de groepsfot...,HNqNQNMXio4,https://www.youtube.com/watch?v=HNqNQNMXio4&t=...,"[3, 71, 15]"
3,u op de,107,109,vervolgens drukt u op de neertoets en de optoe...,DEHw1VDSKVc,https://www.youtube.com/watch?v=DEHw1VDSKVc&t=...,"[33, 17, 3]"
4,u die u,508,509,u niet vreemd over de hitte van de verdrukking...,Co4vE_sSnZM,https://www.youtube.com/watch?v=Co4vE_sSnZM&t=...,"[33, 20, 33]"
...,...,...,...,...,...,...,...
98013,we hebben echt heel veel mensen nodig om dit t...,392,397,we hebben echt heel veel mensen nodig om dit t...,g01V_H0TU8o,https://www.youtube.com/watch?v=g01V_H0TU8o&t=...,"[11, 49, 92, 141, 113, 115, 114, 29, 30, 15, 5..."
98014,die we nu gaan doen maar ik wilde er wel even ...,234,240,die we nu gaan doen maar ik wilde er wel even ...,CqaNlQx_CmE,https://www.youtube.com/watch?v=CqaNlQx_CmE&t=...,"[20, 11, 54, 63, 58, 18, 0, 159, 19, 47, 111, ..."
98015,dus ja ik denk zeker dat het heel anders is en...,265,269,dus ja ik denk zeker dat het heel anders is en...,eldUSfVfYMc,https://www.youtube.com/watch?v=eldUSfVfYMc&t=...,"[88, 48, 0, 90, 146, 4, 2, 141, 167, 5, 8, 0, ..."
98016,dat er voor ja moet er geen gaan we nou ja maa...,694,700,jarenlang dat er voor ja moet er geen gaan we ...,9rsOGla4V6s,https://www.youtube.com/watch?v=9rsOGla4V6s&t=...,"[4, 19, 21, 48, 44, 19, 41, 63, 11, 106, 48, 1..."


In [15]:
# # word usage in max condition
# result_list = []
# index_list = []
# for i in range(len(df_youtube_sent_select)):
#     search = df_youtube_sent_select.loc[i,"search_string"]
#     start=df_youtube_sent_select.loc[i,"start_time"]
#     end=df_youtube_sent_select.loc[i,"end_time"]
#     sent=df_youtube_sent_select.loc[i,"sentence"]
#     id = df_youtube_sent_select.loc[i,"video_id"]
#     id_url = df_youtube_sent_select.loc[i,"video_url"]
#     word_index = df_youtube_sent_select.loc[i,"word_index_list"]
#     list_var = index_list + word_index
#     #list_var = index_list + [x for x in word_index]
# 
#     dict_list_count = Counter(list_var)
#     count_list = list(dict_list_count.values())
# 
#     if any([True if i>word_use_num_max else False for i in count_list]):
#         index_list = index_list
#     else:
#         index_list = index_list + word_index
#         result_list.append([search,start,end,sent,id,id_url,word_index])
#     

In [16]:
# word usage in min max condition
word_num_index_dict = {}
for i in df_word_select["index"]:
    word_num_index_dict[i] = 0

result_list = []
index_list = []
for i in range(len(df_youtube_sent_select)):
    search = df_youtube_sent_select.loc[i,"search_string"]
    start=df_youtube_sent_select.loc[i,"start_time"]
    end=df_youtube_sent_select.loc[i,"end_time"]
    sent=df_youtube_sent_select.loc[i,"sentence"]
    id = df_youtube_sent_select.loc[i,"video_id"]
    id_url = df_youtube_sent_select.loc[i,"video_url"]
    word_index = df_youtube_sent_select.loc[i,"word_index_list"]
    list_var = index_list + word_index
    #list_var = index_list + [x for x in word_index]
    # word count for max
    dict_list_count = Counter(list_var)
    count_list = list(dict_list_count.values())
     # word count for min
    count_list2 = list(word_num_index_dict.values())   

    if any([True if i>word_use_num_max else False for i in count_list]) or not(any([True if j<word_use_num_min else False for j in count_list2])):
        index_list = index_list
    else:
        index_list = index_list + word_index
        result_list.append([search,start,end,sent,id,id_url,word_index])

        for item2 in dict_list_count.items(): 
            word_num_index_dict[item2[0]] = item2[1]

print(f"Max condition: {any([True if i>word_use_num_max else False for i in count_list2])} \nMin condition: {any([True if j<word_use_num_min else False for j in count_list2])}") 

Max condition: False 
Min condition: True


In [17]:
#word_num_index_dict

In [18]:
set_index = set(index_list)
set_all_index = set([x for x in df_word_select["index"]])

In [19]:
not_in_sent = set_all_index.difference(set_index)
not_in_sent = list(not_in_sent)
not_in_sent

[128,
 137,
 148,
 151,
 152,
 154,
 155,
 156,
 157,
 159,
 160,
 165,
 166,
 169,
 171,
 175,
 176,
 177,
 180,
 185,
 187,
 189,
 194,
 196,
 198,
 199,
 74,
 90,
 103,
 107,
 127]

In [20]:
#Counter(index_list)

In [21]:
df_youtube_sent_sample = pd.DataFrame(result_list, columns=["search_string","start_time","end_time","sentence","video_id","video_url","word_index"])
df_youtube_sent_sample

,search_string,start_time,end_time,sentence,video_id,video_url,word_index
0,en n z,446,448,en daarmee speeldn we e oorlogje o e zeeslage ...,P8gWZjlPaNo,https://www.youtube.com/watch?v=P8gWZjlPaNo&t=...,"[8, 26, 139]"
1,ga ik u,40,41,en zoals gezegd in de inleiding ga ik u vandaa...,Gokx4ZpGRC0,https://www.youtube.com/watch?v=Gokx4ZpGRC0&t=40s,"[60, 0, 33]"
2,de m te,253,255,vorig jaar deed de m te vormen op de groepsfot...,HNqNQNMXio4,https://www.youtube.com/watch?v=HNqNQNMXio4&t=...,"[3, 71, 15]"
3,u op de,107,109,vervolgens drukt u op de neertoets en de optoe...,DEHw1VDSKVc,https://www.youtube.com/watch?v=DEHw1VDSKVc&t=...,"[33, 17, 3]"
4,u die u,508,509,u niet vreemd over de hitte van de verdrukking...,Co4vE_sSnZM,https://www.youtube.com/watch?v=Co4vE_sSnZM&t=...,"[33, 20, 33]"
...,...,...,...,...,...,...,...
294,gewoon geen genoeg geld,129,130,maar we krijgen gewoon geen genoeg geld,UvDi1HCeLpc,https://www.youtube.com/watch?v=UvDi1HCeLpc&t=...,"[121, 41, 195, 178]"
295,mensen wordt twee mensen,125,127,vier mensen wordt twee mensen anderhalve meter,eaxYIPwzBAA,https://www.youtube.com/watch?v=eaxYIPwzBAA&t=...,"[115, 135, 124, 115]"
296,willen maken binnen onze,11,13,die een start willen maken binnen onze organis...,Ah_MM39XWhg,https://www.youtube.com/watch?v=Ah_MM39XWhg&t=11s,"[168, 133, 188, 112]"
297,niets anders gedaan moet,460,462,ik heb echter niets anders gedaan moet geluk z...,4YSldeXmuXw,https://www.youtube.com/watch?v=4YSldeXmuXw&t=...,"[109, 167, 142, 44]"


In [22]:
df_youtube_sent_sample["video_url"] = "https://www.youtube.com/watch?v="+df_youtube_sent_sample['video_id'].map(str)+"&t="+df_youtube_sent_sample['start_time'].map(str)+"s"
df_youtube_sent_sample

,search_string,start_time,end_time,sentence,video_id,video_url,word_index
0,en n z,446,448,en daarmee speeldn we e oorlogje o e zeeslage ...,P8gWZjlPaNo,https://www.youtube.com/watch?v=P8gWZjlPaNo&t=...,"[8, 26, 139]"
1,ga ik u,40,41,en zoals gezegd in de inleiding ga ik u vandaa...,Gokx4ZpGRC0,https://www.youtube.com/watch?v=Gokx4ZpGRC0&t=40s,"[60, 0, 33]"
2,de m te,253,255,vorig jaar deed de m te vormen op de groepsfot...,HNqNQNMXio4,https://www.youtube.com/watch?v=HNqNQNMXio4&t=...,"[3, 71, 15]"
3,u op de,107,109,vervolgens drukt u op de neertoets en de optoe...,DEHw1VDSKVc,https://www.youtube.com/watch?v=DEHw1VDSKVc&t=...,"[33, 17, 3]"
4,u die u,508,509,u niet vreemd over de hitte van de verdrukking...,Co4vE_sSnZM,https://www.youtube.com/watch?v=Co4vE_sSnZM&t=...,"[33, 20, 33]"
...,...,...,...,...,...,...,...
294,gewoon geen genoeg geld,129,130,maar we krijgen gewoon geen genoeg geld,UvDi1HCeLpc,https://www.youtube.com/watch?v=UvDi1HCeLpc&t=...,"[121, 41, 195, 178]"
295,mensen wordt twee mensen,125,127,vier mensen wordt twee mensen anderhalve meter,eaxYIPwzBAA,https://www.youtube.com/watch?v=eaxYIPwzBAA&t=...,"[115, 135, 124, 115]"
296,willen maken binnen onze,11,13,die een start willen maken binnen onze organis...,Ah_MM39XWhg,https://www.youtube.com/watch?v=Ah_MM39XWhg&t=11s,"[168, 133, 188, 112]"
297,niets anders gedaan moet,460,462,ik heb echter niets anders gedaan moet geluk z...,4YSldeXmuXw,https://www.youtube.com/watch?v=4YSldeXmuXw&t=...,"[109, 167, 142, 44]"


In [23]:
df_youtube_sent_sample.drop("word_index", axis=1, inplace=True)

In [24]:
df_youtube_sent_sample.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Group_In_Youtube_Sentence_Sample.xlsx", index=False)

##### Selected Result Word Count

In [25]:
#df_selected_file = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment2/Result/6-Youtube Sentence Pos Tag Analysis/Deploy Result Manuel/{lang_folder.capitalize()}_200_Word_Group_In_Youtube_Sentence_Word_Index_List_Sample Revised.xlsx")
df_selected_file = pd.read_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Group_In_Youtube_Sentence_Sample.xlsx")
df_selected_file

,search_string,start_time,end_time,sentence,video_id,video_url
0,en n z,446,448,en daarmee speeldn we e oorlogje o e zeeslage ...,P8gWZjlPaNo,https://www.youtube.com/watch?v=P8gWZjlPaNo&t=...
1,ga ik u,40,41,en zoals gezegd in de inleiding ga ik u vandaa...,Gokx4ZpGRC0,https://www.youtube.com/watch?v=Gokx4ZpGRC0&t=40s
2,de m te,253,255,vorig jaar deed de m te vormen op de groepsfot...,HNqNQNMXio4,https://www.youtube.com/watch?v=HNqNQNMXio4&t=...
3,u op de,107,109,vervolgens drukt u op de neertoets en de optoe...,DEHw1VDSKVc,https://www.youtube.com/watch?v=DEHw1VDSKVc&t=...
4,u die u,508,509,u niet vreemd over de hitte van de verdrukking...,Co4vE_sSnZM,https://www.youtube.com/watch?v=Co4vE_sSnZM&t=...
...,...,...,...,...,...,...
294,gewoon geen genoeg geld,129,130,maar we krijgen gewoon geen genoeg geld,UvDi1HCeLpc,https://www.youtube.com/watch?v=UvDi1HCeLpc&t=...
295,mensen wordt twee mensen,125,127,vier mensen wordt twee mensen anderhalve meter,eaxYIPwzBAA,https://www.youtube.com/watch?v=eaxYIPwzBAA&t=...
296,willen maken binnen onze,11,13,die een start willen maken binnen onze organis...,Ah_MM39XWhg,https://www.youtube.com/watch?v=Ah_MM39XWhg&t=11s
297,niets anders gedaan moet,460,462,ik heb echter niets anders gedaan moet geluk z...,4YSldeXmuXw,https://www.youtube.com/watch?v=4YSldeXmuXw&t=...


In [26]:
def word_count_result(df,column_list): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    
    return  df_word_count

In [27]:
df_shared_word_manuel_count = word_count_result(df_selected_file, df_selected_file.columns[0:1])
df_shared_word_manuel_count

,word,word_count
0,aan,7
1,tot,7
2,over,7
3,kom,7
4,s,7
...,...,...
164,helpen,1
165,haar,1
166,weet,1
167,werk,1


In [28]:
df_shared_word_manuel_count.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Group_In_Youtube_Sentence_Word_Count.xlsx", index=False)

##### Not In Sent And Selected Word List

In [29]:
#lang_folder = "Turkish"
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,ik,22539531
1,je,20769946
2,het,15696161
3,de,15258816
4,dat,13387137
...,...,...
331769,oorlogsgezichten,5
331770,opdrachtenlijst,5
331771,verlsaafde,5
331772,oxidatieproces,5


In [30]:
#word_start = 0
#word_end = 200
df_word_select = df_word_all.iloc[word_start:word_end,]
df_word_select

,word,frequency
0,ik,22539531
1,je,20769946
2,het,15696161
3,de,15258816
4,dat,13387137
...,...,...
195,genoeg,353128
196,wist,352948
197,oh,351988
198,klaar,350506


In [31]:
def index_to_word(index_list, df_word, word_column, reverse=True):
    '''
    index_to_word(not_in_sent, df_word_select, "word", reverse=True)\n
    not in sent is index list as numeric\n
    df_word_select is a dataframe that includes word colunm ("word" is a column)\n
    reverse is used for string index list convert numeric.
    '''
    if reverse:
        word_index_dict = {var1:var2 for (var1, var2) in enumerate(df_word[f"{word_column}"])}
    else:
        word_index_dict = {var2:var1 for (var1, var2) in enumerate(df_word[f"{word_column}"])}

    convert_word_list = []
    for index in index_list:
        convert_word = word_index_dict[index]
        convert_word_list.append(convert_word)
    return convert_word_list        

In [32]:
not_in_sent_word = index_to_word(not_in_sent, df_word_select, "word", reverse=True)
not_in_sent_word

['hou',
 'dood',
 'allemaal',
 'vader',
 'dag',
 'doet',
 'wacht',
 'zie',
 'dacht',
 'wilde',
 'geef',
 'andere',
 'dank',
 'mr',
 'wilt',
 'waren',
 'spijt',
 'niemand',
 'praten',
 'gezien',
 'zitten',
 'moeder',
 'vind',
 'wist',
 'klaar',
 'neem',
 'hebt',
 'denk',
 'misschien',
 'zei',
 'tegen']

In [33]:
set_not_in_sent_word = set(not_in_sent_word)

In [34]:
set_word = set(df_word_select["word"])
set_select_word = set(df_shared_word_manuel_count["word"])

In [35]:
set_not_in_select = set_word.difference(set_select_word)
set_not_in_select

{'allemaal',
 'andere',
 'dacht',
 'dag',
 'dank',
 'denk',
 'doet',
 'dood',
 'geef',
 'gezien',
 'hebt',
 'hou',
 'klaar',
 'misschien',
 'moeder',
 'mr',
 'neem',
 'niemand',
 'praten',
 'spijt',
 'tegen',
 'vader',
 'vind',
 'wacht',
 'waren',
 'wilde',
 'wilt',
 'wist',
 'zei',
 'zie',
 'zitten'}

In [36]:
not_in_sent_selected_word_list = list(set_not_in_select.union(set_not_in_sent_word))
not_in_sent_selected_word_list

['klaar',
 'zei',
 'dood',
 'vader',
 'hebt',
 'andere',
 'wilde',
 'dank',
 'hou',
 'neem',
 'waren',
 'wist',
 'gezien',
 'dag',
 'moeder',
 'praten',
 'niemand',
 'mr',
 'zie',
 'zitten',
 'allemaal',
 'geef',
 'dacht',
 'doet',
 'misschien',
 'denk',
 'vind',
 'wacht',
 'wilt',
 'spijt',
 'tegen']

In [37]:
len(not_in_sent_selected_word_list)

31

In [38]:
df_youtube_sent_select.drop(["word_index_list"], axis=1, inplace=True)
df_youtube_sent_select

/home/kurubal/anaconda3/envs/py39/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,search_string,start_time,end_time,sentence,video_id,video_url
0,en n z,446,448,en daarmee speeldn we e oorlogje o e zeeslage ...,P8gWZjlPaNo,https://www.youtube.com/watch?v=P8gWZjlPaNo&t=...
1,ga ik u,40,41,en zoals gezegd in de inleiding ga ik u vandaa...,Gokx4ZpGRC0,https://www.youtube.com/watch?v=Gokx4ZpGRC0&t=40s
2,de m te,253,255,vorig jaar deed de m te vormen op de groepsfot...,HNqNQNMXio4,https://www.youtube.com/watch?v=HNqNQNMXio4&t=...
3,u op de,107,109,vervolgens drukt u op de neertoets en de optoe...,DEHw1VDSKVc,https://www.youtube.com/watch?v=DEHw1VDSKVc&t=...
4,u die u,508,509,u niet vreemd over de hitte van de verdrukking...,Co4vE_sSnZM,https://www.youtube.com/watch?v=Co4vE_sSnZM&t=...
...,...,...,...,...,...,...
98013,we hebben echt heel veel mensen nodig om dit t...,392,397,we hebben echt heel veel mensen nodig om dit t...,g01V_H0TU8o,https://www.youtube.com/watch?v=g01V_H0TU8o&t=...
98014,die we nu gaan doen maar ik wilde er wel even ...,234,240,die we nu gaan doen maar ik wilde er wel even ...,CqaNlQx_CmE,https://www.youtube.com/watch?v=CqaNlQx_CmE&t=...
98015,dus ja ik denk zeker dat het heel anders is en...,265,269,dus ja ik denk zeker dat het heel anders is en...,eldUSfVfYMc,https://www.youtube.com/watch?v=eldUSfVfYMc&t=...
98016,dat er voor ja moet er geen gaan we nou ja maa...,694,700,jarenlang dat er voor ja moet er geen gaan we ...,9rsOGla4V6s,https://www.youtube.com/watch?v=9rsOGla4V6s&t=...


In [39]:
def word_in_wordgroup(source_word_list, df_target, target_column, word_sample_num, simple=False):

    '''word_in_wordgroup(not_in_sent_word_list, df_youtube_sent_select, "search_string", 5, simple=False)\n
       source_word_list is searching word list\n
       df_target is dataframe, target_column are dataframe column string name\n
       word_sample_num is searching sample number.
       simple use for all column result or only target column result 
    '''
    if simple:
        df_select = df_target[[f"{target_column}"]].dropna()
    else:
        df_select = df_target
        
    df_result = pd.DataFrame()
    for i in source_word_list:
        try:
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].head(word_sample_num)    
        except:
            pass        
        word_in_word_cluster.insert(0,"word",i)
        df_result = pd.concat([df_result,word_in_word_cluster], axis=0)
    df_result.reset_index(drop=True, inplace=True)

    return df_result

In [40]:
if youtube_sent_usage == "all": 
    df_youtube_usage_select = df_youtube_sent_result
else:
    df_youtube_usage_select = df_youtube_sent_select

In [41]:
not_in_sent_selected_word_list_result = word_in_wordgroup(not_in_sent_selected_word_list, df_youtube_usage_select, "search_string", 5, simple=False)
not_in_sent_selected_word_list_result

,word,search_string,start_time,end_time,sentence,video_id,video_url
0,klaar,klaar om de,690,694,we staan klaar om de resterende,JOvs8lz4jMY,https://www.youtube.com/watch?v=JOvs8lz4jMY&t=...
1,klaar,klaar met de,211,214,klaar met de snede en ik zet de bank stil en i...,HqEGHcP413w,https://www.youtube.com/watch?v=HqEGHcP413w&t=...
2,klaar,klaar met de,338,339,wouter maartje ik ben helemaal klaar met de pr...,5Xw_0BqK8so,https://www.youtube.com/watch?v=5Xw_0BqK8so&t=...
3,klaar,klaar om het,445,448,medewerkers van ieder in stonden klaar om het ...,3fAvv1RgbUQ,https://www.youtube.com/watch?v=3fAvv1RgbUQ&t=...
4,klaar,klaar is die,230,232,klaar is die,I5R1UpWMoxY,https://www.youtube.com/watch?v=I5R1UpWMoxY&t=...
...,...,...,...,...,...,...,...
147,tegen,en tegen de,93,94,en tegen de toekomst van je eigen woning,X2ZaLBIXqqQ,https://www.youtube.com/watch?v=X2ZaLBIXqqQ&t=93s
148,tegen,en tegen de,131,133,ondernemen tegen de organisatoren van deze bet...,OtLPqHsFnbg,https://www.youtube.com/watch?v=OtLPqHsFnbg&t=...
149,tegen,tegen in te,500,501,zoiets en jamie al overweegt aanklachten tegen...,rnd3Id2vnp8,https://www.youtube.com/watch?v=rnd3Id2vnp8&t=...
150,tegen,dat ik tegen,31,32,wat mij tegenhield om het gebed te verrichten ...,HdmT36t4rsc,https://www.youtube.com/watch?v=HdmT36t4rsc&t=31s


In [42]:
#not_in_sent_selected_word_list_merge = pd.merge(not_in_sent_selected_word_list_result, df_youtube_sent_select, how="left", on="search_string")
#not_in_sent_selected_word_list_merge.drop_duplicates(inplace=True)
#not_in_sent_selected_word_list_merge

In [43]:
not_in_sent_selected_word_list_result.word.nunique()

31

In [44]:
set_not_in_sent_selected_word_list = set(not_in_sent_selected_word_list)
set_not_in_sent_selected_word_list_result = set(not_in_sent_selected_word_list_result["word"].to_list())

In [45]:
set_not_in_sent_selected_word_list.difference(set_not_in_sent_selected_word_list_result)

set()

In [46]:
#word_in_wordgroup_simple(["pekala","gece"], df_youtube_sent_select, "search_string", 5)

In [47]:
not_in_sent_selected_word_list_result["video_url"] = "https://www.youtube.com/watch?v="+not_in_sent_selected_word_list_result['video_id'].map(str)+"&t="+not_in_sent_selected_word_list_result['start_time'].map(str)+"s"
not_in_sent_selected_word_list_result

,word,search_string,start_time,end_time,sentence,video_id,video_url
0,klaar,klaar om de,690,694,we staan klaar om de resterende,JOvs8lz4jMY,https://www.youtube.com/watch?v=JOvs8lz4jMY&t=...
1,klaar,klaar met de,211,214,klaar met de snede en ik zet de bank stil en i...,HqEGHcP413w,https://www.youtube.com/watch?v=HqEGHcP413w&t=...
2,klaar,klaar met de,338,339,wouter maartje ik ben helemaal klaar met de pr...,5Xw_0BqK8so,https://www.youtube.com/watch?v=5Xw_0BqK8so&t=...
3,klaar,klaar om het,445,448,medewerkers van ieder in stonden klaar om het ...,3fAvv1RgbUQ,https://www.youtube.com/watch?v=3fAvv1RgbUQ&t=...
4,klaar,klaar is die,230,232,klaar is die,I5R1UpWMoxY,https://www.youtube.com/watch?v=I5R1UpWMoxY&t=...
...,...,...,...,...,...,...,...
147,tegen,en tegen de,93,94,en tegen de toekomst van je eigen woning,X2ZaLBIXqqQ,https://www.youtube.com/watch?v=X2ZaLBIXqqQ&t=93s
148,tegen,en tegen de,131,133,ondernemen tegen de organisatoren van deze bet...,OtLPqHsFnbg,https://www.youtube.com/watch?v=OtLPqHsFnbg&t=...
149,tegen,tegen in te,500,501,zoiets en jamie al overweegt aanklachten tegen...,rnd3Id2vnp8,https://www.youtube.com/watch?v=rnd3Id2vnp8&t=...
150,tegen,dat ik tegen,31,32,wat mij tegenhield om het gebed te verrichten ...,HdmT36t4rsc,https://www.youtube.com/watch?v=HdmT36t4rsc&t=31s


In [48]:
not_in_sent_selected_word_list_result.to_excel(f"{lang_folder.capitalize()}_{word_end}_Word_Group_In_Youtube_Sentence_Sample2.xlsx", index=False)

#### Copy Move And Delete

In [49]:
output_file = glob.glob(f"{lang_folder.capitalize()}_{word_end}_Word_Group_In_Youtube_Sentence_*.xlsx")
output_file

['Dutch_200_Word_Group_In_Youtube_Sentence_Sample.xlsx',
 'Dutch_200_Word_Group_In_Youtube_Sentence_Word_Count.xlsx',
 'Dutch_200_Word_Group_In_Youtube_Sentence_Sample2.xlsx']

In [50]:
for k in output_file:
    source = k # source directory
    destination = path
    shutil.copy2(source, destination)

In [51]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass